IGENIERÍA DE DATOS

INGESTA DE DATOS

In [1]:
#Importo la librería pandas y gzip (cabe aclarar que se debe tener instalado pandas previamente, para eso utilizo el comando pip install pandas)
import pandas as pd
import gzip
import numpy as np
import pyarrow as pa

Decido utilizar los 3 archivos directamente en formato comprimido, para el caso del archivo "steam_games", se puede leer directamente utilizando
gzip pero para los siguientes 2 archivos: "user_reviews" y "user_items" hubo que hacer una función ya que el json venía con comillas simples

Ingesta y limpieza de datos del primer archivo "steam_games"

In [2]:
# Con gzip abrimos el json y generamos un dataframe
with gzip.open('/Users/gaston/Documents/Carrera Data Analytics/Proyectos Individuales/Proyecto 1/steam_games.json.gz', 'rt', encoding='utf-8') as steam_games:
       
    df_steam_games = pd.read_json(steam_games, lines=True)

In [3]:
# Decido borrar algunas columnas que no formaban parte del diccionario de datos y que además formaban parte de otro dataset
steam_games = df_steam_games.drop(columns = ['items','user_id','steam_id','items_count','tags'])

In [4]:
# Borro nulos
steam_games.dropna(inplace = True, how='all')

In [5]:
#realizo una función para separar la columna "release_date" en year, month y day para luego consumir en las funciones
def procesar_fecha(fecha):
    if fecha and fecha != 'none':
        partes = fecha.split("-")
        if len(partes) == 3:
            año = partes[0]
            mes = partes[1]
            dia = partes[2]
            return año, mes, dia
    return None, None, None

# Aplico la función a la columna "release_date" y creo columnas separadas para año, mes y día
steam_games[['release_year', 'release_month', 'release_day']] = steam_games['release_date'].apply(lambda x: pd.Series(procesar_fecha(x)))

# Elimino la columna original "release_date"
steam_games.drop(columns=['release_date'], inplace=True)


Ingesta y limpieza de datos del archivo "user_reviews"

In [6]:
#Ahora genero un dataframe para "user_reviews" utilizando una función que recorre y completa una lista vacía con la cual genero el DF
import ast #importo la librería ast a utilizar

info = [] 

for i in gzip.open('/Users/gaston/Documents/Carrera Data Analytics/Proyectos Individuales/Proyecto 1/user_reviews.json.gz'):
     info.append(ast.literal_eval(i.decode('utf-8')))
     
df_user_reviews = pd.DataFrame(info) 


In [7]:
# Con las funciones explode y normalize desanidamos las columna "reviews" para poder
# disponibilizar esos datos para consumirlos en etapas posteriores
df_reviews_exploded = df_user_reviews.explode('reviews')
df_reviews_desanidado = pd.json_normalize(df_reviews_exploded['reviews'].dropna())

# Por otra parte reindexamos los DF
df_reviews_desanidado.reset_index(inplace=True)
df_reviews_exploded.reset_index(inplace=True)

# Concatenamos y eliminamosla columna que originalmente desanidamos

user_reviews = pd.concat([df_reviews_exploded,df_reviews_desanidado], axis=1)
user_reviews = user_reviews.drop(columns = ['reviews'])

In [8]:
#verifico y borro los duplicados
user_reviews_duplicados =  user_reviews.duplicated( keep="first")
user_reviews_duplicados.sum()

10

In [9]:
user_reviews = user_reviews.drop_duplicates(keep = "first")

In [10]:
#realizo una función para separar la columna "posted" en dia, mes y año para luego consumir en las funciones

def procesar_fecha(data):
    import re
    if isinstance(data, str):
        match = re.search(r'(\w+) (\d+), (\d+)', data)
        if match:
            mes = match.group(1)
            dia = match.group(2)
            año = match.group(3)
            meses = {
                'January': '01',
                'February': '02',
                'March': '03',
                'April': '04',
                'May': '05',
                'June': '06',
                'July': '07',
                'August': '08',
                'September': '09',
                'October': '10',
                'November': '11',
                'December': '12'
            }
            mes_numero = meses.get(mes)
            if mes_numero is not None:
                return mes_numero, dia, año
    return None, None, None

# Aplico la función a la columna "posted" después de convertirla en cadenas de texto y unifico la fecha en el formato
#YYYY-MM-DD para consumir en las funciones
user_reviews[['Mes', 'Día', 'Año']] = user_reviews['posted'].astype(str).apply(lambda x: pd.Series(procesar_fecha(x)))
user_reviews['Año'].fillna(0, inplace=True)
user_reviews['Mes'].fillna(0, inplace=True)
user_reviews['Día'].fillna(0, inplace=True)


# Convierto las columnas 'Año', 'Mes' y 'Día' en enteros
user_reviews['Año'] = user_reviews['Año'].astype(int)
user_reviews['Año'] = user_reviews['Año'].fillna(0).astype(int)
user_reviews['Mes'] = user_reviews['Mes'].astype(int)
user_reviews['Mes'] = user_reviews['Mes'].fillna(0).astype(int)
user_reviews['Día'] = user_reviews['Día'].astype(int)
user_reviews['Día'] = user_reviews['Día'].fillna(0).astype(int)

# Defino una fecha predeterminada para las filas con valores cero
fecha_predeterminada = pd.to_datetime('1900-01-01')

# Crea una nueva columna 'fecha_review' con la fecha predeterminada para las filas con valores cero
user_reviews['fecha_review'] = user_reviews.apply(lambda row: pd.to_datetime(f"{row['Año']}-{row['Mes']}-{row['Día']}", format='%Y-%m-%d', errors='coerce') if row['Año'] != 0 and row['Mes'] != 0 and row['Día'] != 0 else fecha_predeterminada, axis=1)

# Elimino la columna original "posted"
user_reviews.drop(columns=['posted'], inplace=True)


Sobre este dataset, realizo el análisis de sentimiento con NLP utilizando la librería NLTK

In [11]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

Al ejecutar estos comandos, me encontré con errores por los certificados SSL, y tuve que instalar de otra forma como
muestro en el siguiente Markdown
import nltk
nltk.download('vader_lexicon')
nltk.download('punkt')

import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

In [12]:
#defino la función para analizar los sentimientos
def analizar_sentimiento(texto):
    if isinstance(texto, str):
        sia = SentimentIntensityAnalyzer()
        sentimiento = sia.polarity_scores(texto)
    
        # Determinar la etiqueta en función de la puntuación compuesta
        if sentimiento['compound'] >= 0.05:
            return '2'
        elif sentimiento['compound'] <= -0.05:
            return '0'
        else:
            return '1'
    else:
            return '1'

In [13]:
# Aplico la fución sobre la columna "review"" y genero una nueva denominada "sentimiento" donde indico con 0 si es negativo
# 1 si es neutral y 2 si es positivo de acuerdo a lo solicitado
user_reviews['sentimiento'] = user_reviews['review'].apply(analizar_sentimiento)

In [14]:
#Borro registros con valores nulos en la columna "review" y duplicados
user_reviews = user_reviews.dropna(subset=['review'])
user_reviews = user_reviews.drop_duplicates( keep = "first")

In [15]:
#Elimino la columma "index"
user_reviews = user_reviews.drop(columns = ['index'])

Ingesta y limpieza de datos del archivo "user_items"

In [16]:
#Ahora genero un dataframe para "users_items" utilizando una función que recorre y completa una lista vacía con la cual genero el DF
import ast #importo la librería ast a utilizar

info = [] 

for i in gzip.open('/Users/gaston/Documents/Carrera Data Analytics/Proyectos Individuales/Proyecto 1/users_items.json.gz'):
     info.append(ast.literal_eval(i.decode('utf-8')))
     
df_users_items = pd.DataFrame(info) 

In [17]:
# Con las funciones explode y normalize desanidamos las columna "items" para poder
# disponibilizar esos datos para consumirlos en etapas posteriores
df_items_exploded = df_users_items.explode('items')
df_items_desanidado = pd.json_normalize(df_items_exploded['items'])

# Por otra parte reindexamos los DF
df_items_exploded.reset_index(inplace=True)
df_items_desanidado.reset_index(inplace=True)

# Concatenamos y eliminamos la columna que previamente desanidamos
user_items = pd.concat([df_items_exploded,df_items_desanidado], axis=1)
user_items = user_items.drop(columns = ['items'])

In [18]:
#identifico valores duplicados, como no hay registros duplicados, no es necesario borrar
user_items_duplicados =  user_items.duplicated( keep="first")
user_items_duplicados.sum()

0

In [19]:
#borro los duplicados
user_items = user_items.drop_duplicates(keep = "first")

Uno los datasets (DataFrames) user_items & steam_games ya que su info es complementaria y necesaria para las funciones
que debemos armar pero primero cambio el tipo de dato a float para poder unirlos

In [20]:
user_items['item_id']=user_items['item_id'].astype(float)

In [ ]:
steam_games['item_id']=user_items['item_id'].astype(float)

In [22]:
user_items_steam_games = user_items.merge(steam_games, left_on='item_id',right_on='id',how='left')

Para la columna precio, defino que si el valor no es numérico, sea Nan para evitar errores de cálculos

In [ ]:
user_items_steam_games['price'] = user_items_steam_games['price'].apply(lambda x: 0 if isinstance(x, str) else (x if pd.notna(x) else np.nan))


TRATAMIENTO DE DATOS

In [24]:
#borro las columnas "index" del dataframe user_items_steam_games
user_items_steam_games = user_items_steam_games.drop(columns = ['index'])

In [25]:
#Al intentar exportar el df a parquet, encontré que en la columna "metascore" tenía valores "none" que
#me generaban error ya que no es un Nan, por lo que decido reemplazarlos por NA

# Reemplazo 'NA' con NaN en la columna 'metascore'
user_items_steam_games['metascore'] = user_items_steam_games['metascore'].replace('NA', np.nan)

In [26]:
#Decido borrar las columnas playtime_2weeks, specs y publisher para reducir espacio en el dataframe user_items_steam_games
user_items_steam_games = user_items_steam_games.drop(columns = ['playtime_2weeks'])
user_items_steam_games = user_items_steam_games.drop(columns = ['specs'])
user_items_steam_games = user_items_steam_games.drop(columns = ['publisher'])

Luego de confeccionados los dataframes con la info que necesito para consumir en las funciones,
decido guardarlos en archivos parquet y no en csv para optimizar 

Para eso primero instalo la biblioteca pyarrow con el comando pip install pyarrow

In [28]:
#renombro los dataframes
reviews= user_reviews
users_games = user_items_steam_games


En los siguientes pasos, acomodo los Datasets en Dataframes específicos para cada función de la API y poder así luego
asegurar el deploy en render.

In [29]:
#genero un archivo parquet con el dataframe "reviews" para utilizar en las funcione de la api
reviews.to_parquet('user_reviews.parquet', engine='pyarrow', compression='snappy')

In [30]:
#genero un dataframe para consumir en la función userdata
df_userdata = users_games[['user_id','price']]
df_userdata.to_parquet('df_userdata.parquet', engine='pyarrow', compression='snappy')

In [31]:
#genero un dataframe para consumir en la función genre_ranking
users_games_genero = users_games[['genres','playtime_forever']]
 #Realizo un explode de la columna 'genres' para poder desanidar la información en distintos registros
users_games_generos = users_games_genero[['genres','playtime_forever']].explode('genres')
    
# Agrupo por género y sumo los valores de "playtime_forever"
agrupado = users_games_generos.groupby('genres')['playtime_forever'].sum().reset_index()

# Ordeno por la suma de "playtime_forever" en orden descendente
agrupado = agrupado.sort_values(by='playtime_forever', ascending=False)
agrupado.to_parquet('generos.parquet', engine='pyarrow', compression='snappy')


In [42]:
#genero un nuevo dataframe con las columnas genre, playtime_forever, user_id y user_url. Lo exporto a parquet para luego consumirlo
df_generos_horas = users_games[['user_id','user_url','genres','playtime_forever']]
#Realizo un explode de la columna 'genres' para poder desanidar la información en distintos registros
df_generos_horas = df_generos_horas.explode('genres')
    
# Agrupo por usuario y sumo las horas de juego para cada usuario
df_generos_horas = df_generos_horas.groupby(['genres','user_id', 'user_url'])['playtime_forever'].sum().reset_index()

df_generos_horas.to_parquet('generos_horas.parquet', engine='pyarrow', compression='snappy')

In [ ]:
df_generos_horas

In [ ]:
df_generos_horas

In [ ]:
#genero un dataframe para consumir en la función developer
df_developer= users_games[['developer','price','release_year','item_id_x']]
df_developer.to_parquet('df_developer.parquet', engine='pyarrow', compression='snappy')

 # Filtro el DataFrame original por desarrollador y seleccionar columnas relevantes
developer_df = users_games[['developer', 'price', 'release_year', 'item_id_x']]

# Calculo la cantidad de ítems gratuitos (Free) por año para todos los desarrolladores
developer_df['items_free_por_anio'] = developer_df['price'].apply(lambda x: 1 if x == 0 else 0)
items_free_por_anio = developer_df.groupby(['developer', 'release_year'])['items_free_por_anio'].sum().reset_index()

# Calculo la cantidad total de ítems por año para todos los desarrolladores
items_totales_por_anio = developer_df.groupby(['developer', 'release_year'])['item_id_x'].nunique().reset_index()

# Exporto el DataFrame a un archivo Parquet
df_developer = items_free_por_anio.merge(items_totales_por_anio, on=['developer', 'release_year'])
df_developer.to_parquet('df_developer.parquet', engine='pyarrow', compression='snappy')

Como último paso de este Notebook, preparo el Dataset steam_games para realizar luego el EDA y el modelo de recomendación

In [6]:
#ajusto la columna price, defino que si el valor no es numérico, sea 0 para evitar errores de cálculos
steam_games['price'] = steam_games['price'].apply(lambda x: 0 if isinstance(x, str) else (x if pd.notna(x) else np.nan))

In [ ]:
steam_games

In [29]:
steam_games = steam_games[['genres','price','early_access','id','release_year','publisher','app_name','title','developer']]

In [ ]:
steam_games

In [20]:
#exporto el DF steam_games para realizar el EDA luego
steam_games.to_parquet('steam_games.parquet', engine='pyarrow', compression='snappy')